In [ ]:
from bokeh import palettes
import panel as pn
from hvplot import hvPlot

import xarray as  xr
import panel as pn
from jinja2 import Environment, FileSystemLoader

pn.extension()

ds = xr.open_dataset('data/great_lakes.nc')

### SigSlot

In [ ]:
class SigSlot(object):
    """Signal-slot mixin, for Panel event passing"""

    def __init__(self):
        self._sigs = {}
        self._map = {}

    def _register(self, widget, name, thing='value'):
        """Watch the given attribute of a widget and assign it a named event

        This is normally called at the time a widget is instantiated, in the
        class which owns it.

        Parameters
        ----------
        widget : pn.layout.Panel or None
            Widget to watch. If None, an anonymous signal not associated with
            any widget.
        name : str
            Name of this event
        thing : str
            Attribute of the given widget to watch
        """
        self._sigs[name] = {'widget': widget, 'callbacks': [], 'thing': thing}
        wn = "-".join([widget.name if widget is not None else "none", thing])
        self._map[wn] = name
        if widget is not None:
            widget.param.watch(self._signal, thing, onlychanged=True)

    @property
    def signals(self):
        """Known named signals of this class"""
        return list(self._sigs)

    def connect(self, name, callback):
        """Associate call back with given event

        The callback must be a function which takes the "new" value of the
        watched attribute as the only parameter. If the callback return False,
        this cancels any further processing of the given event.
        """
        self._sigs[name]['callbacks'].append(callback)

    def _signal(self, event):
        wn = "-".join([event.obj.name, event.name])
#         print(wn, event)
        if wn in self._map and self._map[wn] in self._sigs:
            self._emit(self._map[wn], event.new)

    def _emit(self, sig, value=None):
        for callback in self._sigs[sig]['callbacks']:
            if callback(value) is False:
                break

    def show(self):
        self.panel.show()

### Xrviz DataArray/Coordinate Selection Panel 

In [ ]:
class DataSelector(SigSlot):
    """
    This widget takes input as a xarray instance.
    For each dataset the properties:
    'Dimensions','Coordinates','Variables','Attributes'
    are dislpayed. On selecting one of these,the expansion 
    occurs to show  the arrtibutes associated with it.
    
    It of these individual property could be expanded accordingly.
    
    Parameters
    ----------
    ds: `xarray` instance: `DataSet` or `DataArray`
        datset is used to initialize the DataSelector
        
    Attributes
    ----------
    selected_property: property which has been  selected in multiselect
        One of 'Dimensions','Coordinates','Variables','Attributes'
    selected_subproperty: subproperty which has been selected
        An attribute of its property.
        Example: `nx`,`ny`,`time` for property `Dimensions`.
    panel: Displays the generated Multiselect object
          
    Reason for adding initial letter in front of a sub_property:
       It will act as option separator for the multiSelect.
       Ex: Variable 'time' could be present in both `Dimensions` and
           and `Coordinates`. Upon selection of any one, both are
           automatically selected. However in presence of `letter`
           `d :time` belongs to dimensions while `c : time` belongs
           to `Coordinates`.
           
    """
    
    def __init__(self,ds):
        super().__init__()
        if isinstance(ds,xr.Dataset) or isinstance(ds,xr.DataArray):
            self.data = ds
        
        self.down = '│'
        self.right = '└──'
        self.properties = {'Dimensions' :'dims',
                           'Coordinates':'coords',
                           'Variables'  :'data_vars',
                           'Attributes' :'attrs'}
        
        self.rev_map = {'d' : 'Dimensions',
                        'c' : 'Coordinates',
                        'v' : 'Variables',
                        'a' : 'Attributes'}
            
        self.selected_property = None
        self.selected_subproperty = None      
        self.select =  pn.widgets.MultiSelect(size=8, min_width=300, width_policy='min')
        self.fill_items(self.select.options)
                
        self._register(self.select,"property_selector")
        self.connect("property_selector",self.expand_or_collapse_nested)
         
        self.panel = pn.Row(self.select)
    
    def fill_items(self,items):
        if isinstance(self.data,xr.Dataset):
            items.append('# DataSet')
            for prop in list(self.properties):
                items.append(f'{self.right} {prop}')
        else:
            items.append('# DataArray')
        
    def expand_or_collapse_nested(self,value):
        self._property = value[0].split()[-1]
        
        def find_prop_and_subprop(value):
            prop,subprop = None,None
            if " : " in value[0]:
                subprop = value[0].split()[-1]
                prop = self.rev_map[value[0].split()[-3]]
            else:
                prop = value[0].split()[-1]
            return prop,subprop
        
        def collapse(options,index):
            for opt in options[index+1:]:
                if opt.split()[0] ==self.right:
                    break
                if self.down in opt:
                    options.remove(opt)
            return options
        
        def expand(options,name,index):
            children  = get_children(name)
            for i, child in enumerate(children):
                options.insert(index+i+1, (f'{self.down} {self.right} {child}'))
            return options
        
        def get_children(prop):
            ds_prop = self.properties[prop]
            if ds_prop == "data_vars":
                return ['v'+ " : " + child for child in getattr(ds,str(ds_prop)).keys()] 
            else:
                return [ds_prop[0]+ " : " + child for child in getattr(ds,str(ds_prop)).keys()]                 
        
        def has_expanded(selected_property,index):
            next_index = index+1
            if (next_index) < len(old) and (self.down in old[next_index]):
                return True
            else: 
                return False
                                   
        self.selected_property, self.selected_subproperty = find_prop_and_subprop(value)
            
        if self._property in list(self.properties):
            old = list(self.select.options)
            index,name = next((i,self.selected_property) for i,v in enumerate(old) if self.selected_property in v)
            
            if has_expanded(self.selected_property,index):
                old = collapse(old,index)
                self.select.options = list(old)
            else:
                old = expand(old,name,index)
                self.select.options = list(old)

In [ ]:
# d  = DataSelector(ds)
# d.panel

In [ ]:
class Description(SigSlot):
    def __init__(self,ds):
        super().__init__()
        self.ds = ds
        self.panel = pn.pane.HTML(style={'font-size': '12pt'},width = 400) 
        self.panel.object = "Description Area"
        self.template_env = Environment(loader=FileSystemLoader('templates'))
        self.variable_template = self.template_env.get_template('variable_info.html')
    
    def variable_pane(self,var):
        variable_attributes = [(k,v) for k,v in self.ds[var].attrs.items()]
        output = self.variable_template.render(variable_attributes = variable_attributes)
#         pane = pn.pane.HTML(output, 
#                             style={'background-color': '#F6F6F6', 'border': '2px solid black',
#                                     'border-radius': '5px', 'padding': '10px','width':'300px'})

        return output
    
    def setup(self,selected_property,sub_property):
        if selected_property == 'Variables':
            if sub_property != None:
                self.panel.object = self.variable_pane(sub_property)
            else:
                self.panel.object = str(selected_property) + " : "+str(sub_property)
                
        else:
            self.panel.object = str(selected_property) + " : "+str(sub_property)           


#     def setup(self,selected_property,sub_property):
#         if selected_property == 'Variables':
#             if sub_property == 'None':
#                 self.panel.object = str(selected_property) + " : "+str(sub_property) 
#             else:
#                 self.panel = self.variable_pane('lat')
                

#         else:
#             self.panel.object = str(selected_property) + " : "+str(sub_property)        

In [ ]:
des = Description(ds)
des.panel

In [ ]:
class Control(SigSlot):
    def __init__(self,ds):
        super().__init__()
        self.selector = DataSelector(ds)
        self.describer = Description(ds)
        
        self._register(self.selector.select,"selection")

        self.connect("selection", self.callback)
        
        self.panel = pn.Row(self.selector.panel,self.describer.panel)
        
    def callback(self,_):
        selected_property = self.selector.selected_property
        sub_property = self.selector.selected_subproperty
        self.describer.setup(selected_property,sub_property)

In [ ]:
c = Control(ds)
c.panel.show()